# Homework.

In this homework, we'll deploy the Straight vs Curly Hair Type model we trained in the previous homework.

Download the model files from here:

- https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data
- https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx

### Imports and setup.
Run this cell first to install dependencies (if needed) and import libraries.

In [21]:
%pip install numpy pillow onnx onnxruntime requests

import numpy as np
import onnx
import onnxruntime as ort
from PIL import Image
from io import BytesIO
from urllib import request
import os

print("Libraries imported successfully!")


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Libraries imported successfully!


### Constants and Configuration
Define the URLs and filenames used.

In [22]:
# URLs
MODEL_URL = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx"
WEIGHTS_URL = "https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle/hair_classifier_v1.onnx.data"
IMAGE_URL = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"

# Local Paths
MODEL_PATH = os.path.join('models', 'hair_classifier_v1.onnx')
WEIGHTS_PATH = os.path.join('models', 'hair_classifier_v1.onnx.data') # Must be in same folder as model
IMAGE_PATH = os.path.join('data', 'image.jpeg')

# Target size from Previous Homework
TARGET_SIZE = (200, 200)

### Helper Functions
Functions to download files, process images, and prepare the input vector.

In [23]:
def download_file(url, save_path):
    if not os.path.exists(save_path):
        print(f"Downloading to {save_path}...")
        request.urlretrieve(url, save_path)
    else:
        print(f"{save_path} already exists.")

def prepare_image(img_path, target_size):
    img = Image.open(img_path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_input(img):
    # Load as float32
    x = np.array(img, dtype='float32')
    x /= 255.0
    
    # Define mean and std specifically as float32 to avoid upcasting to double
    mean = np.array([0.485, 0.456, 0.406], dtype='float32')
    std = np.array([0.229, 0.224, 0.225], dtype='float32')
    
    x = (x - mean) / std
    x = x.transpose((2, 0, 1))
    x = np.expand_dims(x, axis=0)
    
    # Final safety cast to ensure it is strictly float32
    return x.astype('float32')

## Question 1 (Output Node Name)
Downloads the model and inspects the graph to find the output node name.

In [24]:
# 1. Download Model and Weights
download_file(MODEL_URL, MODEL_PATH)
download_file(WEIGHTS_URL, WEIGHTS_PATH)

# 2. Question 1: Load model and get output name
model = onnx.load(MODEL_PATH)
output_name = model.graph.output[0].name

print(f"\n--- Question 1 ---")
print(f"Output node name: {output_name}")

models/hair_classifier_v1.onnx already exists.
models/hair_classifier_v1.onnx.data already exists.

--- Question 1 ---
Output node name: output


## Prepare Image & Question 2, 3
Downloads the image and processes it.

In [25]:
# 1. Download Image
download_file(IMAGE_URL, IMAGE_PATH)

# 2. Process Image
img_prepared = prepare_image(IMAGE_PATH, TARGET_SIZE)
X_input = preprocess_input(img_prepared)

# 3. Question 3: Get first pixel of R channel
r_pixel_value = X_input[0, 0, 0, 0]

print(f"\n--- Question 2 ---")
print(f"Target size used: {TARGET_SIZE}")

print(f"\n--- Question 3 ---")
print(f"Value in first pixel (R channel): {r_pixel_value:.3f}")

data/image.jpeg already exists.

--- Question 2 ---
Target size used: (200, 200)

--- Question 3 ---
Value in first pixel (R channel): -1.073


## Question 4.
Runs the model using the files in the model/ folder.

In [26]:
# Initialize Session using the organized path
session = ort.InferenceSession(MODEL_PATH)

input_name = session.get_inputs()[0].name
outputs = session.run([output_name], {input_name: X_input})
prediction = outputs[0][0][0]

print(f"\n--- Question 4 ---")
print(f"Model output: {prediction:.2f}")


--- Question 4 ---
Model output: 0.09


## Create the Lambda Function Code
Run this cell to create the lambda_function.py file.

In [27]:
%%writefile lambda_function.py
import numpy as np
import onnxruntime as ort
from PIL import Image
from io import BytesIO
from urllib import request
import os

# Configuration matching the internal docker model
MODEL_FILE = "hair_classifier_empty.onnx"
TARGET_SIZE = (200, 200)

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_input(img):
    x = np.array(img, dtype='float32')
    x /= 255.0
    mean = np.array([0.485, 0.456, 0.406], dtype='float32')
    std = np.array([0.229, 0.224, 0.225], dtype='float32')
    x = (x - mean) / std
    x = x.transpose((2, 0, 1))
    x = np.expand_dims(x, axis=0)
    return x.astype('float32')

# Global session to load model only once (standard Lambda practice)
session = None

def init_session():
    global session
    if session is None:
        # Check if the model exists in the container
        if not os.path.exists(MODEL_FILE):
             raise FileNotFoundError(f"Model file {MODEL_FILE} not found. Are you running inside the correct Docker container?")
        session = ort.InferenceSession(MODEL_FILE)

def predict(url):
    init_session()
    
    # 1. Download and Process
    img = download_image(url)
    img_prepared = prepare_image(img, TARGET_SIZE)
    X = preprocess_input(img_prepared)
    
    # 2. Inference
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    
    result = session.run([output_name], {input_name: X})
    return float(result[0][0][0])

def lambda_handler(event, context):
    url = event['url']
    result = predict(url)
    return result

Overwriting lambda_function.py


### Create the Dockerfile
Run this cell to create the Dockerfile that defines the container environment.

In [31]:
%%writefile Dockerfile
FROM agrigorev/model-2025-hairstyle:v1

# Install required libraries
RUN pip install numpy onnxruntime pillow

# Copy our lambda function code into the container
COPY lambda_function.py .

# Set the CMD to your handler
CMD [ "lambda_function.lambda_handler" ]

Overwriting Dockerfile


## Question 6.

### Test Script.
Docker cointainer must be running.
```
# 1. Build your custom image
docker build -t homework-hair .

# 2. Run the container (mapping port 8080)
docker run -it --rm -p 8080:8080 homework-hair
```

The cell will create 'test.py', then you can run it in the terminal as follows:

```
python test.py
```

In [39]:
%%writefile test.py
import requests

url = "http://localhost:8080/2015-03-31/functions/function/invocations"
data = {"url": "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"}

try:
    response = requests.post(url, json=data)
    print("Response status:", response.status_code)
    print("Response text:", response.text)
    print(f"\n--- Question 6 Answer ---")
    print(f"Model output: {response.text}")
except Exception as e:
    print("Error: Could not connect to Docker. Make sure the container is running in your terminal!")
    print(e)

Overwriting test.py


In [40]:
!python test.py

Response status: 200
Response text: -0.10220836102962494

--- Question 6 Answer ---
Model output: -0.10220836102962494
